# Wells Distance Join - Appetizer 🥗

**Purpose**: Join domestic wells distance data to the TMK foundation
**Input**: TMK_Foundation_Master + Domestic Wells distance data
**Output**: Enhanced foundation with both municipal and domestic wells distances

## Recipe Overview
Building on our TMK foundation (which already has municipal wells), we add domestic wells distance data. This completes the wells setback requirements for HAR 11-62 compliance.

### Key Ingredients:
- TMK_Foundation_Master (from 01a)
- Domestic Wells distance shapefile
- HAR 11-62 setback requirements (1000 ft for all wells)

### Cooking Steps:
1. Load TMK foundation from previous step
2. Import domestic wells distance data
3. Join on TMK field
4. Standardize field names for Matrix compatibility
5. Update JOIN_LOG tracking

In [ ]:
# Load master configuration if not already loaded
if 'outputs_folder' not in globals():
    exec(open(r"../00_Master_Menu/00_Master_Hawaii_Matrix_Analysis.ipynb").read())

import sys
sys.path.append(os.path.join(scripts_folder, '99_Kitchen_Utils'))
from Common_Functions import *

print("🥗 Starting Wells Distance Join...")
log_workflow_step("Wells Distance Join", "Beginning domestic wells join")

In [ ]:
# Verify TMK Foundation exists from previous step
tmk_folder = os.path.join(outputs_folder, "TMK_Foundation")
tmk_foundation_path = os.path.join(tmk_folder, "TMK_Foundation_Master.shp")

if not os.path.exists(tmk_foundation_path):
    print("❌ TMK Foundation not found! Run 01a_TMK_Foundation_Setup first.")
    raise FileNotFoundError(f"Missing: {tmk_foundation_path}")

print(f"✅ TMK Foundation found: {tmk_foundation_path}")

# Check current record count
foundation_count = int(arcpy.management.GetCount(tmk_foundation_path)[0])
print(f"Foundation records: {foundation_count:,}")

In [ ]:
# Locate domestic wells shapefile
domestic_wells_shp = os.path.join(data_folder, "gis_downloads", "wells", "statewide", 
                                  "CPs_Distance_to_Domestic_Wells.shp")

if not os.path.exists(domestic_wells_shp):
    print(f"❌ Domestic wells shapefile not found: {domestic_wells_shp}")
    raise FileNotFoundError("Download domestic wells data from Dr. Shuler's portal")

print(f"✅ Domestic wells found: {domestic_wells_shp}")

# Analyze domestic wells structure
domestic_fields = [f.name for f in arcpy.ListFields(domestic_wells_shp)]
domestic_count = int(arcpy.management.GetCount(domestic_wells_shp)[0])

print(f"Domestic wells records: {domestic_count:,}")
print(f"TMK fields found: {[f for f in domestic_fields if 'TMK' in f.upper()]}")
print(f"Distance fields: {[f for f in domestic_fields if 'dist' in f.lower() or 'near' in f.lower()]}")

In [ ]:
# Import domestic wells to geodatabase for join
domestic_wells_fc = "Domestic_Wells_Distance_Import"
arcpy.management.CopyFeatures(domestic_wells_shp, domestic_wells_fc)
print(f"✅ Imported to geodatabase: {domestic_wells_fc}")

# Identify TMK join fields
foundation_fields = [f.name for f in arcpy.ListFields(tmk_foundation_path)]
domestic_gdb_fields = [f.name for f in arcpy.ListFields(domestic_wells_fc)]

# Find TMK field (prioritize TMK, then TMK9, then TMK_txt)
tmk_candidates = ['TMK', 'TMK9', 'TMK_txt']
foundation_tmk = None
domestic_tmk = None

for candidate in tmk_candidates:
    if candidate in foundation_fields:
        foundation_tmk = candidate
        break

for candidate in tmk_candidates:
    if candidate in domestic_gdb_fields:
        domestic_tmk = candidate
        break

if not foundation_tmk or not domestic_tmk:
    print(f"❌ TMK field mismatch:")
    print(f"   Foundation: {[f for f in foundation_fields if 'TMK' in f.upper()]}")
    print(f"   Domestic: {[f for f in domestic_gdb_fields if 'TMK' in f.upper()]}")
    raise ValueError("Cannot identify matching TMK fields")

print(f"✅ TMK join fields: {foundation_tmk} ←→ {domestic_tmk}")

In [ ]:
# Perform the join
print("🔄 Joining domestic wells data...")

try:
    # Use JoinField for attribute join
    arcpy.management.JoinField(
        in_data=tmk_foundation_path,
        in_field=foundation_tmk,
        join_table=domestic_wells_fc,
        join_field=domestic_tmk
    )
    
    print("✅ Join completed successfully!")
    
    # Verify join results
    post_join_fields = [f.name for f in arcpy.ListFields(tmk_foundation_path)]
    new_fields = [f for f in post_join_fields if f not in foundation_fields]
    
    print(f"New fields added: {len(new_fields)}")
    for field in new_fields:
        print(f"  + {field}")
        
except Exception as e:
    print(f"❌ Join failed: {e}")
    raise

In [ ]:
# Standardize field names for Matrix compatibility
print("🔄 Standardizing field names...")

current_fields = [f.name for f in arcpy.ListFields(tmk_foundation_path)]

# Field renaming mapping (adjust based on actual field names)
field_renames = {
    'NEAR_DIST': 'Dist_Municipal_Wells_ft',  # May already be renamed
    'NEAR_DIST_1': 'Dist_Domestic_Wells_ft',
    'Distance': 'Dist_Municipal_Wells_ft',
    'Distance_1': 'Dist_Domestic_Wells_ft'
}

renamed_count = 0
for old_name, new_name in field_renames.items():
    if old_name in current_fields:
        try:
            arcpy.management.AlterField(
                in_table=tmk_foundation_path,
                field=old_name,
                new_field_name=new_name,
                new_field_alias=new_name
            )
            print(f"  ✅ Renamed: {old_name} → {new_name}")
            renamed_count += 1
        except Exception as e:
            print(f"  ⚠️ Could not rename {old_name}: {e}")

print(f"✅ Standardized {renamed_count} field names")

In [ ]:
# Update JOIN_LOG tracking
print("📝 Updating join log...")

timestamp = datetime.now().strftime('%Y-%m-%d %H:%M')

with arcpy.da.UpdateCursor(tmk_foundation_path, ["JOIN_LOG"]) as cursor:
    for row in cursor:
        current_log = row[0] or ""
        new_log = f"{current_log}; Wells: {timestamp}"
        row[0] = new_log[:200]  # Truncate to field length
        cursor.updateRow(row)

print("✅ Join log updated")

In [ ]:
# Final validation and summary
final_count = int(arcpy.management.GetCount(tmk_foundation_path)[0])
final_fields = [f.name for f in arcpy.ListFields(tmk_foundation_path)]

# Look for distance fields
distance_fields = [f for f in final_fields if 'dist' in f.lower() and 'well' in f.lower()]

print("\n" + "="*50)
print("🎉 WELLS DISTANCE JOIN COMPLETE!")
print("="*50)
print(f"TMK Foundation: {tmk_foundation_path}")
print(f"Records: {final_count:,} (should match original {foundation_count:,})")
print(f"Total fields: {len(final_fields)}")
print(f"Distance fields: {distance_fields}")
print(f"\nReady for next step: Soil HAR 11-62 Classification")

# Clean up temporary import
if arcpy.Exists(domestic_wells_fc):
    arcpy.management.Delete(domestic_wells_fc)
    print(f"🗑️ Cleaned up temporary import: {domestic_wells_fc}")

log_workflow_step("Wells Distance Join", "Complete - domestic wells joined")